# Catanatron Introduction
This shows example usage of Catanatron. First, clone the repo and install requirements.

In [2]:

# !pip install -r catanatron/dev-requirements.txt
# !pip install -e catanatron/catanatron_core
# !pip install -e catanatron/catanatron_server
# !pip install -e catanatron/catanatron_gym
# !pip install -e catanatron/catanatron_experimental
# exit() # Forcefully restart runtime to picks up installed requirements

Obtaining file:///home/alex/Masters/catanatron/catanatron_experimental/catanatron_experimental/catanatron/catanatron_core
  Preparing metadata (setup.py) ... done
  Attempting uninstall: catanatron
    Found existing installation: catanatron 3.2.1
    Uninstalling catanatron-3.2.1:
      Successfully uninstalled catanatron-3.2.1
  Running setup.py develop for catanatron
Obtaining file:///home/alex/Masters/catanatron/catanatron_experimental/catanatron_experimental/catanatron/catanatron_experimental
  Preparing metadata (setup.py) ... done
  Attempting uninstall: catanatron_experimental
    Found existing installation: catanatron_experimental 1.0.0
    Uninstalling catanatron_experimental-1.0.0:
      Successfully uninstalled catanatron_experimental-1.0.0
  Running setup.py develop for catanatron_experimental


Then, you can implement your own bot strategy and pit against some benchmark bots.

In [1]:
import random

from catanatron.game import Game
from catanatron.models.player import Player, RandomPlayer, Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron.players.search import VictoryPointPlayer

class MyPlayer(Player):
    def decide(self, game, playable_actions):
        """Should return one of the playable_actions.

        Args:
            game (Game): complete game state. read-only.
            playable_actions (Iterable[Action]): options to choose from
        Return:
            action (Action): Chosen element of playable_actions
        """
        # ===== YOUR CODE HERE =====
        # As an example we simply choose a valid action at random:
        return random.choice(playable_actions)
        # ===== END YOUR CODE =====

# Play a simple 4v4 game. Edit MyPlayer with your logic!
players = [
    MyPlayer(Color.RED),
    WeightedRandomPlayer(Color.BLUE),
    VictoryPointPlayer(Color.WHITE),
    RandomPlayer(Color.ORANGE),
]
game = Game(players)
print(game.play())  # returns winning color

Color.WHITE


You can also simulate thousands of games to get more statistically significant results:

In [8]:
from pprint import pprint
from catanatron_experimental.play import play_batch

wins, results_by_player, games = play_batch(20, players)

Output()

                                      Last 10 Games                                      
     ╷                       ╷       ╷        ╷         ╷          ╷           ╷         
   # │ SEATING               │ TURNS │ RED VP │ BLUE VP │ WHITE VP │ ORANGE VP │ WINNER  
╶────┼───────────────────────┼───────┼────────┼─────────┼──────────┼───────────┼────────╴
  11 │ ORANGE,WHITE,RED,BLUE │   235 │      3 │       8 │       11 │         3 │ WHITE   
  12 │ ORANGE,RED,WHITE,BLUE │   335 │     10 │       4 │        3 │         5 │ RED     
  13 │ BLUE,ORANGE,WHITE,RED │   172 │      2 │       4 │       10 │         3 │ WHITE   
  14 │ ORANGE,BLUE,RED,WHITE │   195 │      2 │      10 │        9 │         6 │ BLUE    
  15 │ RED,WHITE,BLUE,ORANGE │   119 │      3 │       2 │       10 │         2 │ WHITE   
  16 │ BLUE,RED,ORANGE,WHITE │   366 │      8 │      10 │        2 │         9 │ BLUE    
  17 │ WHITE,BLUE,ORANGE,RED │   167 │      3 │      10 │        4 │         2 │ BLUE    
  18 │ RED,BLUE,WHITE,ORANGE │   245 │      2 │       3 │        7 │        10 │ ORANGE  
  19 │ ORANGE,RED,BLUE,WHITE │   184 │      4 │      11 │        4 │         2 │ BLUE    
  20 │ RED,WHITE,BLUE,ORANGE │   240 │      3 │      10 │        6 │         5 │ BLUE    
     ╵                       ╵       ╵        ╵         ╵          ╵           ╵

                                              Player Summary                                               
                            ╷      ╷        ╷             ╷            ╷          ╷          ╷             
                            │ WINS │ AVG VP │ AVG SETTLES │ AVG CITIES │ AVG ROAD │ AVG ARMY │ AVG DEV VP  
╶───────────────────────────┼──────┼────────┼─────────────┼────────────┼──────────┼──────────┼────────────╴
  MyPlayer:RED              │    2 │   4.20 │        1.75 │       0.55 │     0.10 │     0.25 │       0.65  
  WeightedRandomPlayer:BLUE │    6 │   5.80 │        1.90 │       1.00 │     0.05 │     0.35 │       1.10  
  VictoryPointPlayer:WHITE  │    8 │   7.05 │        2.50 │       0.95 │     0.45 │     0.15 │       1.45  
  RandomPlayer:ORANGE       │    4 │   5.30 │        2.15 │       0.65 │     0.40 │     0.20 │       0.65  
                            ╵      ╵        ╵             ╵            ╵          ╵          ╵

              Game Summary              
            ╷           ╷               
  AVG TICKS │ AVG TURNS │ AVG DURATION  
╶───────────┼───────────┼──────────────╴
     650.80 │    223.30 │   0.109 secs  
            ╵           ╵

You can inspect the game states in a variety of ways and compute statistics

In [11]:
from catanatron.json import GameEncoder
from catanatron_gym.features import create_sample_vector, create_sample
from catanatron_gym.board_tensor_features import (
    create_board_tensor,
)

game = games[-1]  # pick say the first one
# pprint(game)
# 1. Feature dictionary of last state before game ended from REDs perspective. 
#   See https://catanatron.readthedocs.io/en/latest/catanatron_gym.envs.html#catanatron_gym.envs.catanatron_env.CatanatronEnv.observation_space
#   for more information on this representation.
# record = create_sample(game, Color.RED)
# pprint(record)

# 2. Vector (similar to 1) of last state before game ended
# vector = create_sample_vector(game, Color.RED)
# print(vector)

# 3. Board Tensor representation, similar to the one described in https://arxiv.org/abs/2008.07079
# tensor = create_board_tensor(game, Color.RED)
# print(tensor.shape)
# print(tensor)

# 4. Inspect Python catanatron.state.State class
# pprint(vars(game.state))

# 5. JSON Representation (with full action history)
game_json = GameEncoder().default(game)
pprint(game_json)  # inspect a game state representation

{'actions': [Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=26),
             Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(25, 26)),
             Action(color=<Color.WHITE: 'WHITE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=30),
             Action(color=<Color.WHITE: 'WHITE'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(29, 30)),
             Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=0),
             Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(0, 20)),
             Action(color=<Color.ORANGE: 'ORANGE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=36),
             Action(color=<Color.ORANGE: 'ORANGE'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(35, 36)),
             Action(color=<Color.ORANGE: '

## Testing The Random Player Behavours

In [1]:
import random
from pprint import pprint
from catanatron_experimental.play import play_batch
from catanatron.game import Game
from catanatron.models.player import Player, RandomPlayer, Color
from catanatron.players.weighted_random import WeightedRandomPlayer, CityRandomPlayer, SettlementRandomPlayer, LongestRoadRandomPlayer, DevCardRandomPlayer 
from catanatron.players.search import VictoryPointPlayer
from typing import Literal

In [2]:
# Play a simple 4v4 game. Edit MyPlayer with your logic!
players = [
    CityRandomPlayer(Color.RED),
    SettlementRandomPlayer(Color.BLUE),
    LongestRoadRandomPlayer(Color.WHITE),
    DevCardRandomPlayer(Color.ORANGE),
]
game = Game(players,seed=42)
print(game.play())

Color.ORANGE


In [3]:
my_game_config = {
    "discard_limit": 7,
    "vps_to_win": 10,
    "catan_map": "BASE",
    # "map_type":"BASE"
}

wins, results_by_player, games = play_batch(40, players)

Output()

                                      Last 10 Games                                      
     ╷                       ╷       ╷        ╷         ╷          ╷           ╷         
   # │ SEATING               │ TURNS │ RED VP │ BLUE VP │ WHITE VP │ ORANGE VP │ WINNER  
╶────┼───────────────────────┼───────┼────────┼─────────┼──────────┼───────────┼────────╴
  31 │ BLUE,WHITE,ORANGE,RED │   373 │     10 │       7 │        6 │         9 │ RED     
  32 │ WHITE,BLUE,ORANGE,RED │   205 │     10 │       5 │        3 │         8 │ RED     
  33 │ WHITE,RED,ORANGE,BLUE │   181 │      2 │      10 │        4 │         7 │ BLUE    
  34 │ RED,ORANGE,WHITE,BLUE │   220 │      3 │       4 │       10 │         3 │ WHITE   
  35 │ RED,ORANGE,WHITE,BLUE │   447 │      9 │       5 │        6 │        10 │ ORANGE  
  36 │ ORANGE,WHITE,BLUE,RED │   416 │      7 │      10 │        2 │         7 │ BLUE    
  37 │ RED,ORANGE,WHITE,BLUE │   607 │      9 │       4 │        9 │        10 │ ORANGE  
  38 │ ORANGE,WHITE,BLUE,RED │   264 │      4 │      10 │        4 │         5 │ BLUE    
  39 │ BLUE,RED,ORANGE,WHITE │   213 │      6 │       4 │       10 │         2 │ WHITE   
  40 │ WHITE,ORANGE,RED,BLUE │   380 │     10 │       6 │        5 │         2 │ RED     
     ╵                       ╵       ╵        ╵         ╵          ╵           ╵

                                                Player Summary                                                 
                                ╷      ╷        ╷             ╷            ╷          ╷          ╷             
                                │ WINS │ AVG VP │ AVG SETTLES │ AVG CITIES │ AVG ROAD │ AVG ARMY │ AVG DEV VP  
╶───────────────────────────────┼──────┼────────┼─────────────┼────────────┼──────────┼──────────┼────────────╴
  CityRandomPlayer:RED          │   11 │   5.72 │        1.68 │       1.77 │     0.25 │     0.00 │       0.00  
  SettlementRandomPlayer:BLUE   │   10 │   5.95 │        2.80 │       1.35 │     0.23 │     0.00 │       0.00  
  LongestRoadRandomPlayer:WHITE │    9 │   5.88 │        2.58 │       0.35 │     0.38 │     0.35 │       1.15  
  DevCardRandomPlayer:ORANGE    │   10 │   6.22 │        1.93 │       0.70 │     0.15 │     0.50 │       1.60  
                                ╵      ╵        ╵             ╵            ╵          ╵          ╵

              Game Summary              
            ╷           ╷               
  AVG TICKS │ AVG TURNS │ AVG DURATION  
╶───────────┼───────────┼──────────────╴
     959.83 │    325.23 │   0.048 secs  
            ╵           ╵

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
